# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: ` https://country-leaders.herokuapp.com/status``
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [6]:
# import the requests library (1 line)
import requests
    
# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.herokuapp.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = root_url + "/status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)


"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [7]:
# Set the country_url variable (1 line)
country_url = root_url + "/countries"

# query the /country endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(country_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json() #data from API is trurned in JSON

# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)

403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [8]:
# Set the cookie_url variable (1 line)
cookie_url = root_url + "/cookie"

# Query the enpoint, set the cookies variable and display it (3 lines)
req = requests.get(cookie_url)
cookie = req.cookies
print(cookie)

<RequestsCookieJar[<Cookie user_cookie=19e11567-c140-4257-9cde-9a29e3e9e17c for country-leaders.herokuapp.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [9]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
req = requests.get(country_url, cookies = cookie)

# and check the status code and the countries variable's content (3 lines)
countries = req.json()
print(req.status_code)
print(countries)



200
['us', 'fr', 'ma', 'ru', 'be']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [10]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
req = requests.get(root_url + "/leaders", cookies = cookie)

# and check its content (4 lines)
print(req.json())

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.herokuapp.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [11]:
# Set the leaders_url variable (1 line)
leaders_url = root_url + "/leaders"
# query the /leaders endpoint using cookies and parameters (take any country in countries)

req = requests.get(leaders_url, cookies = cookie, params={'country':countries[0]})

# check the result (3 lines) 
print(req)

<Response [200]>


### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [12]:
# 5 lines
leaders_per_country = {}
for country in countries:
    req = requests.get(leaders_url, cookies=cookie, params={"country": country})
    leaders_per_country[country] = req.json()
    print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [13]:
# 15 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    leaders_url = root_url + "/leaders"
    cookie_url = root_url + "/cookie"
    countries_url = root_url + "/countries"
    
    req_cookies = requests.get(cookie_url)
    cookies = req_cookies.cookies
    #print(cookies)

    req_countries = requests.get(countries_url, cookies=cookies)
    countries = req_countries.json()
    #print(countries)

    leaders_per_country = {}

    for country in countries:
        req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        #print(req)
        leaders_per_country[country] = req.json()
    return leaders_per_country


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [14]:
# 2 lines
leaders_per_country = get_leaders()
print(leaders_per_country)

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'plac

In [15]:
# 3 lines
# req_cookies = requests.get(cookie_url)
# cookies = req_cookies.cookies


req = requests.get("https://nl.wikipedia.org/wiki/Herman_Van_Rompuy", cookies = cookie)
leader_wikipedia_html = req.text
print(leader_wikipedia_html)

<!DOCTYPE html>
<html class="client-nojs" lang="nl" dir="ltr">
<head>
<meta charset="UTF-8"/>
<title>Herman Van Rompuy - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"86af8b5d-eeee-4376-929b-f782e279c20f","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Herman_Van_Rompuy","wgTitle":"Herman Van Rompuy","wgCurRevisionId":62717884,"wgRevisionId":62717884,"wgArticleId":113729,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CD\u0026V-politicus","Partijvoorzitter van de CVP (België)","Voorzitter van de Kamer van volksvertegenwoordigers","Belgisch senator","Mi

Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [16]:
# 3 lines

import bs4
soup = bs4.BeautifulSoup(leader_wikipedia_html , 'html.parser')
# print(soup.get_text())
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="nl">
 <head>
  <meta charset="utf-8"/>
  <title>
   Herman Van Rompuy - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t.",".\t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","januari","februari","maart","april","mei","juni","juli","augustus","september","oktober","november","december"],"wgRequestId":"86af8b5d-eeee-4376-929b-f782e279c20f","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Herman_Van_Rompuy","wgTitle":"Herman Van Rompuy","wgCurRevisionId":62717884,"wgRevisionId":62717884,"wgArticleId":113729,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["CD\u0026V-politicus","Partijvoorzitter van de CVP (België)","Voorzitter van de Kamer van volksvertegenwoordigers","Bel

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [17]:
paragraphs = soup.find_all('p')
print(paragraphs)

[<p><b>Herman Achille graaf Van Rompuy</b> (<a href="/wiki/Etterbeek" title="Etterbeek">Etterbeek</a>, <a href="/wiki/31_oktober" title="31 oktober">31 oktober</a> <a href="/wiki/1947" title="1947">1947</a>) is een <a class="mw-redirect" href="/wiki/Belgi%C3%AB_(hoofdbetekenis)" title="België (hoofdbetekenis)">Belgische</a> politicus van de christendemocratische partij <a href="/wiki/Christen-Democratisch_en_Vlaams" title="Christen-Democratisch en Vlaams">CD&amp;V</a>. Hij was tussen 30 december 2008 en 25 november 2009 de <a href="/wiki/Premier_van_Belgi%C3%AB" title="Premier van België">premier van België</a> en van 1 januari 2010<sup class="reference" id="cite_ref-2"><a href="#cite_note-2">[2]</a></sup> tot en met 30 november 2014 <a href="/wiki/Voorzitter_van_de_Europese_Raad" title="Voorzitter van de Europese Raad">voorzitter van de Europese Raad</a>, in de media vaak incorrect '<a href="/wiki/President_van_de_Europese_Unie" title="President van de Europese Unie">president van Eur

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [18]:
# 6 lines
for paragraph in paragraphs:
    if paragraph.find('b'):
        first_paragraph = paragraph.text
        break

print(first_paragraph)

Herman Achille graaf Van Rompuy (Etterbeek, 31 oktober 1947) is een Belgische politicus van de christendemocratische partij CD&V. Hij was tussen 30 december 2008 en 25 november 2009 de premier van België en van 1 januari 2010[2] tot en met 30 november 2014 voorzitter van de Europese Raad, in de media vaak incorrect 'president van Europa' genoemd.[3] Op 19 november 2009 werd bekendgemaakt dat hij was verkozen tot de eerste permanente voorzitter en op 1 maart 2012 werd Van Rompuy met algemene stemmen door de 27 staatshoofden en regeringsleiders van de EU voor een tweede mandaat benoemd. Na deze termijn was hij niet meer herkiesbaar.



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [19]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph
# 2 lines
import bs4 

def get_first_paragraph(wikipedia_url):
    req_cookies = requests.get("https://country-leaders.herokuapp.com/cookie")
    
    req = requests.get(wikipedia_url, cookies = req_cookies.cookies)

    soup = bs4.BeautifulSoup(req.text , 'html.parser')

    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            return paragraph.text


print(get_first_paragraph("https://nl.wikipedia.org/wiki/Herman_Van_Rompuy"))

Herman Achille graaf Van Rompuy (Etterbeek, 31 oktober 1947) is een Belgische politicus van de christendemocratische partij CD&V. Hij was tussen 30 december 2008 en 25 november 2009 de premier van België en van 1 januari 2010[2] tot en met 30 november 2014 voorzitter van de Europese Raad, in de media vaak incorrect 'president van Europa' genoemd.[3] Op 19 november 2009 werd bekendgemaakt dat hij was verkozen tot de eerste permanente voorzitter en op 1 maart 2012 werd Van Rompuy met algemene stemmen door de 27 staatshoofden en regeringsleiders van de EU voor een tweede mandaat benoemd. Na deze termijn was hij niet meer herkiesbaar.



### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [20]:
# 4 lines
import re
print(re.sub(r"\[.\]", "", get_first_paragraph("https://nl.wikipedia.org/wiki/Herman_Van_Rompuy")))


Herman Achille graaf Van Rompuy (Etterbeek, 31 oktober 1947) is een Belgische politicus van de christendemocratische partij CD&V. Hij was tussen 30 december 2008 en 25 november 2009 de premier van België en van 1 januari 2010 tot en met 30 november 2014 voorzitter van de Europese Raad, in de media vaak incorrect 'president van Europa' genoemd. Op 19 november 2009 werd bekendgemaakt dat hij was verkozen tot de eerste permanente voorzitter en op 1 maart 2012 werd Van Rompuy met algemene stemmen door de 27 staatshoofden en regeringsleiders van de EU voor een tweede mandaat benoemd. Na deze termijn was hij niet meer herkiesbaar.



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [21]:
# 10 lines
import bs4
import re

def get_first_paragraph(wikipedia_url):
    req = requests.get(wikipedia_url)

    soup = bs4.BeautifulSoup(req.text , 'html.parser')

    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            return re.sub(r"\[.\]", "", paragraph.text)


print(get_first_paragraph("https://fr.wikipedia.org/wiki/Sadi_Carnot_(homme_d%27%C3%89tat)"))

#https://en.wikipedia.org/wiki/James_A._Garfield
#https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
#https://ru.wikipedia.org/wiki/%D0%95%D0%BB%D1%8C%D1%86%D0%B8%D0%BD,_%D0%91%D0%BE%D1%80%D0%B8%D1%81_%D0%9D%D0%B8%D0%BA%D0%BE%D0%BB%D0%B0%D0%B5%D0%B2%D0%B8%D1%87
#https://fr.wikipedia.org/wiki/Sadi_Carnot_(homme_d%27%C3%89tat)

Marie François Sadi Carnot, plus souvent appelé Sadi Carnot, est un homme d'État français, né le 11 août 1837 à Limoges et mort le 25 juin 1894 à Lyon. Il est président de la République du 3 décembre 1887 à sa mort.



Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [22]:
# < 20 lines


### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [23]:
# < 20 lines
import bs4
import re
import functools

@functools.lru_cache(maxsize=None)
def get_first_paragraph(wikipedia_url):    
    req = requests.get(wikipedia_url)

    soup = bs4.BeautifulSoup(req.text , 'html.parser')

    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            return re.sub(r"\[.\]", "", paragraph.text)


print(get_first_paragraph("https://fr.wikipedia.org/wiki/Sadi_Carnot_(homme_d%27%C3%89tat)"))

Marie François Sadi Carnot, plus souvent appelé Sadi Carnot, est un homme d'État français, né le 11 août 1837 à Limoges et mort le 25 juin 1894 à Lyon. Il est président de la République du 3 décembre 1887 à sa mort.



Let's check the speedup!

In [24]:
%%timeit
# Just run the cell. 
wikipedia_url = "https://fr.wikipedia.org/wiki/Sadi_Carnot_(homme_d%27%C3%89tat)"
get_first_paragraph(wikipedia_url)

58.3 ns ± 0.169 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [25]:
# 19 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    leaders_url = root_url + "/leaders"
    cookie_url = root_url + "/cookie"
    countries_url = root_url + "/countries"
    
    req_cookies = requests.get(cookie_url)
    cookies = req_cookies.cookies
    #print(cookies)

    req_countries = requests.get(countries_url, cookies=cookies)
    countries = req_countries.json()
    #print(countries)

    leaders_per_country = {}

    for country in countries:
        req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        leaders_json = req.json()
        # print(leaders_json)

        leaders_per_country[country] = []

        for leader_dictionary in leaders_json:
            wiki_first_paragraph = get_first_paragraph(leader_dictionary['wikipedia_url'])
            # print(wiki_first_paragraph)

            leader_dictionary['wikipedia_first_paragraph'] = wiki_first_paragraph

            leaders_per_country[country].append(leader_dictionary)

    return leaders_per_country



In [26]:
# Check the output of your function (2 lines)
print(get_leaders())

TypeError: string indices must be integers

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [27]:
# 22 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    leaders_url = root_url + "/leaders"
    cookie_url = root_url + "/cookie"
    countries_url = root_url + "/countries"
    
    req_cookies = requests.get(cookie_url)
    cookies = req_cookies.cookies
    #print(cookies)

    req_countries = requests.get(countries_url, cookies=cookies)
    countries = req_countries.json()
    #print(countries)

    leaders_per_country = {}

    for country in countries:
        req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        
        if(req.status_code != 200):
            req_cookies = requests.get(cookie_url)
            cookies = req_cookies.cookies
            req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        
        leaders_json = req.json()
        # print(leaders_json)

        leaders_per_country[country] = []

        for leader_dictionary in leaders_json:
            wiki_first_paragraph = get_first_paragraph(leader_dictionary['wikipedia_url'])
            # print(wiki_first_paragraph)

            leader_dictionary['wikipedia_first_paragraph'] = wiki_first_paragraph

            leaders_per_country[country].append(leader_dictionary)

    return leaders_per_country




Check the output of your function again.

In [28]:
%%time
# Check the output of your function (1 line)
print(get_leaders())

{'us': [{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04', 'wikipedia_first_paragraph': 'George Washington (February 22, 1732\xa0– December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led the Patriot forces to victory in the American Revolutionary War and served as the president of the Constitutional Convention of 1787, which created the Constitution of the United States and the American federal government. Washington has been called the "Father of the Nation" for his manifold leadership in the formative days of the country.[10]\n'}, {'id': 'Q76', 'first_name': '

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [29]:
# <20 lines
import bs4
import re
import functools

@functools.lru_cache(maxsize=None)
def get_first_paragraph(wikipedia_url, session):   
    req = session.get(wikipedia_url)

    soup = bs4.BeautifulSoup(req.text , 'html.parser')

    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            return re.sub(r"\[.\]", "", paragraph.text)



Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [30]:
# 22 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    leaders_url = root_url + "/leaders"
    cookie_url = root_url + "/cookie"
    countries_url = root_url + "/countries"
    
    req_cookies = requests.get(cookie_url)
    cookies = req_cookies.cookies
    #print(cookies)

    req_countries = requests.get(countries_url, cookies=cookies)
    countries = req_countries.json()
    #print(countries)

    session = requests.Session()

    leaders_per_country = {}

    for country in countries:
        req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        
        if(req.status_code != 200):
            req_cookies = requests.get(cookie_url)
            cookies = req_cookies.cookies
            req = requests.get(leaders_url, cookies = cookies, params = {"country" : country})
        
        leaders_json = req.json()
        # print(leaders_json)

        leaders_per_country[country] = []

        for leader_dictionary in leaders_json:
            wiki_first_paragraph = get_first_paragraph(leader_dictionary['wikipedia_url'], session)

            leader_dictionary['wikipedia_first_paragraph'] = wiki_first_paragraph

            leaders_per_country[country].append(leader_dictionary)

    return leaders_per_country


Test your new functions.



In [31]:
%%time
# Just run it.
leaders_per_country = get_leaders()

CPU times: user 22.9 s, sys: 179 ms, total: 23 s
Wall time: 27.8 s


Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [ ]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [ ]:
# <20 lines
import bs4
import re
import functools

@hashable_cache
def get_first_paragraph(wikipedia_url, session):   

    req = session.get(wikipedia_url)

    soup = bs4.BeautifulSoup(req.text , 'html.parser')

    for paragraph in soup.find_all('p'):
        if paragraph.find('b'):
            return re.sub(r"\[.\]", "", paragraph.text)

Check that the cache is working again.

In [ ]:
%%time
# Just run it.
leaders_per_country = get_leaders()

CPU times: user 160 ms, sys: 11.5 ms, total: 171 ms
Wall time: 1.49 s


## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [ ]:
# 3 lines
import json

filename = "leaders.json"
with open(filename, "w") as my_file:
    my_file.write(json.dumps(leaders_per_country))

Make sure the file can be read back. Write the code to read the file.

In [ ]:
#3 lines
filename = "leaders.json"

with open(filename, "r") as my_file:
    print(my_file.read())

{"us": [{"id": "Q23", "first_name": "George", "last_name": "Washington", "birth_date": "1732-02-22", "death_date": "1799-12-14", "place_of_birth": "Westmoreland County", "wikipedia_url": "https://en.wikipedia.org/wiki/George_Washington", "start_mandate": "1789-04-30", "end_mandate": "1797-03-04", "wikipedia_first_paragraph": "George Washington (February 22, 1732\u00a0\u2013 December 14, 1799) was an American military officer, statesman, and Founding Father who served as the first president of the United States from 1789 to 1797. Appointed by the Continental Congress as commander of the Continental Army, Washington led the Patriot forces to victory in the American Revolutionary War and served as the president of the Constitutional Convention of 1787, which created the Constitution of the United States and the American federal government. Washington has been called the \"Father of the Nation\" for his manifold leadership in the formative days of the country.[10]\n"}, {"id": "Q76", "first

Make a function `save()` to call this code easily.

In [ ]:
# 3 lines
import json
def save():
    filename = "leaders.json"
    with open(filename, "w") as my_file:
        my_file.write(json.dumps(leaders_per_country))

In [ ]:
# Call the function (1 line)
save()

NameError: name 'leaders_per_country' is not defined

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!